In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 14718668
paper_name = 'giaever_davis_2004' 

In [103]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [104]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_csv('raw_data/analyzed_data_hom.txt', sep='\t')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 5918 x 11


In [7]:
original_data.head()

,orf,gene,cisplatin_125_1,cisplatin_125_2,cisplatin_125_3,cisplatin_250_1,cisplatin_250_2,cisplatin_500_1,cisplatin_500_2,cisplatin_66_1,cisplatin_66_2
0,YAL001C,TFC3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,YAL002W,VPS8,6.435788,7.618749,3.775775,7.083423,8.708809,6.785060,7.797910,8.222179,8.407304
2,YAL003W,EFB1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,YAL004W,YAL004W,9.072408,8.489710,10.388901,6.866116,7.557835,7.939483,3.695684,8.638723,10.603179
4,YAL005C,SSA1,7.695610,9.398894,9.857931,6.768785,5.622647,7.994116,8.934242,9.257118,8.550701


In [8]:
original_data['orf'] = original_data['orf'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [orf, gene, cisplatin_125_1, cisplatin_125_2, cisplatin_125_3, cisplatin_250_1, cisplatin_250_2, cisplatin_500_1, cisplatin_500_2, cisplatin_66_1, cisplatin_66_2]
Index: []


In [12]:
original_data.set_index('orf', inplace=True)

In [14]:
original_data.drop(columns=['gene'], inplace=True)

In [16]:
original_data = original_data.apply(pd.to_numeric, errors='coerce')

In [17]:
original_data = original_data.groupby(original_data.index).mean()

In [18]:
original_data.shape

(5890, 9)

# Clean-up

In [21]:
eliminated = pd.read_csv('raw_data/background_tags_base_hom.txt', sep='\t', header=None)

In [22]:
eliminated.head()

,0
0,YAL034W-A
1,YAL046C
2,YAL048C
3,YBL016W
4,YBL025W


In [23]:
eliminated['orf'] = eliminated[0].astype(str)

In [25]:
eliminated['orf'] = clean_orf(eliminated['orf'])

In [26]:
eliminated['orf'] = translate_sc(eliminated['orf'], to='orf')

In [27]:
# Make sure everything translated ok
t = looks_like_orf(eliminated['orf'])
print(eliminated.loc[~t,])

Empty DataFrame
Columns: [0, orf]
Index: []


In [28]:
eliminated_orfs = eliminated['orf'].unique()

In [29]:
original_data.drop(index=eliminated_orfs, inplace=True)

In [32]:
# Remove all ORFs that only have 0s because >90% of them are essential genes

n = original_data.sum(axis=1)

In [35]:
original_data = original_data.loc[n > 0,:]

In [36]:
original_data.shape

(4679, 9)

In [37]:
original_data.columns = [5316, 5316, 5316, 5317, 5317, 5318, 5318, 5313, 5313]

In [38]:
original_data = original_data.T
original_data = original_data.groupby(original_data.index).mean()
original_data = original_data.T

# Load HET data

In [51]:
original_data2 = pd.read_csv('raw_data/analyzed_data_het.txt', sep='\t')

In [52]:
print('Original data dimensions: %d x %d' % (original_data2.shape))

Original data dimensions: 5918 x 73


In [53]:
original_data2.head()

,orf,gene,5-fluorouracil_19.2_1,5-fluorouracil_19.2_2,5-fluorouracil_19.2_3,5-fluorouracil_19.2_4,5-fluorouracil_19.2_5,5-fluorouracil_38.5_1,5-fluorouracil_38.5_2,5-fluorouracil_4.8_1,...,miconazole_0.025_1,miconazole_0.025_2,miconazole_0.05_1,miconazole_0.05_2,miconazole_0.05_3,miconazole_0.05_4,miconazole_0.1_1,miconazole_0.1_2,miconazole_0.2_1,miconazole_0.2_2
0,YMR056C,AAC1,9.526545,7.951773,8.721741,8.346657,4.263601,8.825512,2.692640,7.968219,...,9.312516,9.294373,7.704134,8.931694,9.090973,7.850144,8.901877,7.779176,9.336565,8.923584
1,YBR085W,AAC3,6.453701,10.765296,6.320682,10.383714,11.965008,11.045340,11.969012,11.909970,...,11.644741,11.920223,9.542753,11.970307,11.867074,11.882815,11.451223,11.232877,11.883097,11.970710
2,YNL331C,AAD14,6.524357,6.718113,7.730328,7.950801,6.449246,8.021753,3.075224,5.572534,...,8.802488,8.655241,8.794877,2.469489,8.717330,8.556588,8.616406,8.332019,5.491288,6.489976
3,YCR107W,AAD3,7.457117,8.633749,6.251683,11.965718,11.913022,11.289882,11.435796,11.533248,...,12.027601,8.473162,11.823576,11.998384,12.089701,12.018176,11.883272,12.089652,12.094551,12.062554
4,YDL243C,AAD4,2.168064,6.792825,6.338006,3.879814,9.186689,0.965466,8.789847,7.873645,...,9.092628,8.845137,3.284954,8.006100,8.802386,8.060388,8.540803,9.041727,5.251110,6.069802


In [54]:
original_data2['orf'] = original_data2['orf'].astype(str)

In [56]:
# Eliminate all white spaces & capitalize
original_data2['orf'] = clean_orf(original_data2['orf'])

In [57]:
# Translate to ORFs 
original_data2['orf'] = translate_sc(original_data2['orf'], to='orf')

In [58]:
# Make sure everything translated ok
t = looks_like_orf(original_data2['orf'])
print(original_data2.loc[~t,])

Empty DataFrame
Columns: [orf, gene, 5-fluorouracil_19.2_1, 5-fluorouracil_19.2_2, 5-fluorouracil_19.2_3, 5-fluorouracil_19.2_4, 5-fluorouracil_19.2_5, 5-fluorouracil_38.5_1, 5-fluorouracil_38.5_2, 5-fluorouracil_4.8_1, 5-fluorouracil_76.9_1, 5-fluorouracil_76.9_2, 5-fluorouracil_9.6_1, alverine-citrate_500_1, alverine-citrate_500_2, alverine-citrate_500_3, alverine-citrate_500_4, atorvastatin_125_1, atorvastatin_125_2, atorvastatin_62.5_1, atorvastatin_62.5_2, atorvastatin_62.5_3, cisplatin_125_1, cisplatin_125_2, cisplatin_125_3, cisplatin_31.25_1, cisplatin_31.25_2, cisplatin_62.5_1, cisplatin_62.5_2, dyclonine_250_1, dyclonine_500_1, dyclonine_500_2, dyclonine_500_3, dyclonine_500_4, fenpropimorph_1.17_1, fenpropimorph_2.34_1, fenpropimorph_2.34_2, fenpropimorph_4.69_1, fenpropimorph_4.69_2, fluconazole_130.6_1, fluconazole_130.6_2, fluconazole_16.3_1, fluconazole_32.6_1, fluconazole_32.6_2, fluconazole_65.3_1, fluconazole_65.3_2, lovastatin_154.5_1, lovastatin_154.5_2, lovastatin_

In [59]:
original_data2.set_index('orf', inplace=True)

In [60]:
original_data2.drop(columns=['gene'], inplace=True)

In [61]:
original_data2 = original_data2.apply(pd.to_numeric, errors='coerce')

In [62]:
original_data2 = original_data2.groupby(original_data2.index).mean()

In [63]:
original_data2.shape

(5890, 71)

# Clean-up

In [64]:
eliminated = pd.read_csv('raw_data/background_tags_base.txt', sep='\t', header=None)

In [65]:
eliminated.head()

,0
0,YAL046C
1,YAL048C
2,YBL075C
3,YBR041W
4,YBR075W


In [66]:
eliminated['orf'] = eliminated[0].astype(str)

In [67]:
eliminated['orf'] = clean_orf(eliminated['orf'])

In [68]:
eliminated['orf'] = translate_sc(eliminated['orf'], to='orf')

In [69]:
# Make sure everything translated ok
t = looks_like_orf(eliminated['orf'])
print(eliminated.loc[~t,])

Empty DataFrame
Columns: [0, orf]
Index: []


In [70]:
eliminated_orfs = eliminated['orf'].unique()

In [71]:
original_data2.drop(index=eliminated_orfs, inplace=True)

In [72]:
het_dataset_ids1 = [5312, 5312, 5312, 5312, 5312, 5314, 5314, 5310, 5315, 5315, 5311, 5337, 5337, 5337, 5337, 5323, 5323, 5322, 5322, 5322]
het_dataset_ids2 = [5321, 5321, 5321, 5319, 5319, 5320, 5320, 5338, 5339, 5339, 5339, 5339]
het_dataset_ids3 = [5334, 5335, 5335, 5336, 5336, 5333, 5333, 5330, 5331, 5331, 5332, 5332, 5325, 5325, 5325, 5324]
het_dataset_ids4 = [484, 484, 484, 5308, 5308, 5308, 5308, 5308, 5308, 5308, 5308, 5308, 5309]
het_dataset_ids5 = [5326, 5326, 5327, 5327, 5327, 5327, 5328, 5328, 5329, 5329]

het_dataset_ids = het_dataset_ids1 + het_dataset_ids2 + het_dataset_ids3 + het_dataset_ids4 + het_dataset_ids5

In [73]:
original_data2.columns = het_dataset_ids

In [74]:
original_data2 = original_data2.T
original_data2 = original_data2.groupby(original_data2.index).mean()
original_data2 = original_data2.T

# Merge

In [76]:
original_data = original_data.join(original_data2, how='outer')

In [90]:
# Taking the opposite to make sure that low number indicate low growth and viceversa
original_data = -original_data

# Prepare the final dataset

In [105]:
data = original_data.copy()

In [106]:
dataset_ids = original_data.columns.values
datasets = datasets.reindex(index=dataset_ids)

In [107]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [108]:
data.head()

dataset_id,5313,5316,5317,5318,484,5308,5309,5310,5311,5312,...,5330,5331,5332,5333,5334,5335,5336,5337,5338,5339
data_type,value,value,value,value,value,value,value,value,value,value,...,value,value,value,value,value,value,value,value,value,value
orf,,,,,,,,,,,,,,,,,,,,,
YAL001C,NaN,NaN,NaN,NaN,-10.241723,-8.394513,-12.963801,-5.305541,-12.777055,-14.862464,...,-12.756595,-12.744340,-37.104401,-6.220064,-11.965642,-12.169580,-10.486425,-11.234607,-11.504931,-12.550038
YAL002W,-8.314742,-5.943437,-7.896116,-7.291485,-4.606831,-4.655540,-5.151632,-10.826237,-11.320897,-7.855313,...,-11.360089,-10.073073,-6.074381,-7.628480,-11.216772,-11.303706,-0.000000,-7.683093,-7.473625,-8.000328
YAL003W,NaN,NaN,NaN,NaN,-11.399370,-11.383999,-11.846263,-11.431457,-11.093273,-14.452351,...,-12.447660,-11.964288,-16.708642,-9.905243,-9.511714,-5.222946,-0.000000,-11.625859,-12.559666,-11.406546
YAL004W,-9.620951,-9.317006,-7.211976,-5.817583,-10.543800,-12.249503,-10.459395,-12.291015,-0.000000,-8.455324,...,-12.228788,-9.156718,-5.819805,-4.800479,-9.804812,-10.205156,-4.260552,-12.303574,-9.858953,-12.693315
YAL005C,-8.903910,-8.984145,-6.195716,-8.464179,-9.711878,-7.926802,-10.909778,-10.883547,-7.499103,-8.548105,...,-10.373533,-9.685622,-10.111358,-2.222977,-8.846109,-9.656358,-13.498609,-16.329477,-9.055622,-7.412484


## Subset to the genes currently in SGD

In [109]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 24


In [110]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,5313,5316,5317,5318,484,5308,5309,5310,5311,5312,...,5330,5331,5332,5333,5334,5335,5336,5337,5338,5339
,data_type,value,value,value,value,value,value,value,value,value,value,...,value,value,value,value,value,value,value,value,value,value
gene_id,orf,,,,,,,,,,,,,,,,,,,,,
1,YAL001C,NaN,NaN,NaN,NaN,-10.241723,-8.394513,-12.963801,-5.305541,-12.777055,-14.862464,...,-12.756595,-12.744340,-37.104401,-6.220064,-11.965642,-12.169580,-10.486425,-11.234607,-11.504931,-12.550038
2,YAL002W,-8.314742,-5.943437,-7.896116,-7.291485,-4.606831,-4.655540,-5.151632,-10.826237,-11.320897,-7.855313,...,-11.360089,-10.073073,-6.074381,-7.628480,-11.216772,-11.303706,-0.000000,-7.683093,-7.473625,-8.000328
3,YAL003W,NaN,NaN,NaN,NaN,-11.399370,-11.383999,-11.846263,-11.431457,-11.093273,-14.452351,...,-12.447660,-11.964288,-16.708642,-9.905243,-9.511714,-5.222946,-0.000000,-11.625859,-12.559666,-11.406546
1863,YAL004W,-9.620951,-9.317006,-7.211976,-5.817583,-10.543800,-12.249503,-10.459395,-12.291015,-0.000000,-8.455324,...,-12.228788,-9.156718,-5.819805,-4.800479,-9.804812,-10.205156,-4.260552,-12.303574,-9.858953,-12.693315
4,YAL005C,-8.903910,-8.984145,-6.195716,-8.464179,-9.711878,-7.926802,-10.909778,-10.883547,-7.499103,-8.548105,...,-10.373533,-9.685622,-10.111358,-2.222977,-8.846109,-9.656358,-13.498609,-16.329477,-9.055622,-7.412484


# Normalize

In [111]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [112]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [113]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,5313,5316,5317,5318,484,5308,5309,5310,5311,5312,...,5330,5331,5332,5333,5334,5335,5336,5337,5338,5339
,data_type,value,value,value,value,value,value,value,value,value,value,...,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,,,,,,,,,,,,
1,YAL001C,NaN,NaN,NaN,NaN,-10.241723,-8.394513,-12.963801,-5.305541,-12.777055,-14.862464,...,-1.887700,-0.523506,-1.520588,0.149969,-0.694591,-0.621363,-0.172332,-0.311926,-0.785159,-1.026414
2,YAL002W,-8.314742,-5.943437,-7.896116,-7.291485,-4.606831,-4.655540,-5.151632,-10.826237,-11.320897,-7.855313,...,-1.124286,-0.154045,-0.027042,0.036571,-0.531683,-0.535954,0.282120,0.002433,0.648787,0.446637
3,YAL003W,NaN,NaN,NaN,NaN,-11.399370,-11.383999,-11.846263,-11.431457,-11.093273,-14.452351,...,-1.718818,-0.415617,-0.538894,-0.146741,-0.160766,0.063841,0.282120,-0.346558,-1.160331,-0.656188
1863,YAL004W,-9.620951,-9.317006,-7.211976,-5.817583,-10.543800,-12.249503,-10.459395,-12.291015,-0.000000,-8.455324,...,-1.599169,-0.027305,-0.014789,0.264266,-0.224527,-0.427595,0.097480,-0.406545,-0.199680,-1.072803
4,YAL005C,-8.903910,-8.984145,-6.195716,-8.464179,-9.711878,-7.926802,-10.909778,-10.883547,-7.499103,-8.548105,...,-0.584975,-0.100457,-0.221351,0.471792,-0.015971,-0.373463,-0.302872,-0.762894,0.086067,0.636962


# Print out

In [114]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [115]:
from IO.save_data_to_db3 import *

In [116]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/33 [00:00<?, ?it/s]

Deleting all datasets for PMID 14718668...
Inserting the new data...


100%|██████████| 33/33 [04:43<00:00,  8.60s/it]

Updating the data_modified_on field...
